# Arrhenius with 6 strain rates

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("mime")
import h5py
from scipy.optimize import curve_fit
import lmfit

colors = ['#bb0000', '#00bb00', "#0000bb", '#bbbb00', '#bb00bb', "#00bbbb", '#bbbbbb', '#770000', '#007700', "#000077", '#777700', '#770077', "#007777", '#777777', '#440000', '#004400', "#000044", '#444400', '#440044', "#0044444", '#444444','#000000']

In [ ]:
baseSize = (8, 6)  # Base size of a subplot

def sbPlot(n):
    if (n == 1): return 1, 1
    if (n <= 2): return 1, 2
    if (n <= 4): return 2, 2
    if (n <= 6): return 3, 2
    if (n <= 9): return 3, 3
    if (n <= 12): return 4, 3
    return 0, 0

def sbPlotSize(n):
    x, y = sbPlot(n)
    return baseSize[0] * y, baseSize[1] * x

In [ ]:
h5f = h5py.File('../GleebleData.h5','r')
allData = h5f['all'][:]
shortData = h5f['short'][:]
h5f.close()

In [ ]:
allData.shape, shortData.shape

Remove first point of each curve, where $\varepsilon^p=0$

In [ ]:
allData = allData[allData[:,0] != 0]
shortData = shortData[shortData[:,0]!=0]
identData = shortData

In [ ]:
strains = np.unique(identData[:,0])
allStrains = np.unique(allData[:,0])
epsps = np.unique(identData[:,1])
temperatures = np.unique(identData[:,2])
nEps = len(strains)
nEpsp = len(epsps)
nTemp = len(temperatures)
#strains, epsps, temperatures, nEps, nEpsp, nTemp

# Identification of the Arrhenius parameters
$\sigma^y\left(\varepsilon,\dot\varepsilon,T \right) = \frac{1}{\alpha(\varepsilon)} \ln\left\{\left(\frac{\dot\varepsilon \exp{\left(\frac{Q(\varepsilon)}{RT}\right)}}{A(\varepsilon)}\right)^{\frac{1}{n(\varepsilon)}} + \left[1 + \left(\frac{\dot\varepsilon \exp{\left(\frac{Q(\varepsilon)}{RT}\right)}}{A(\varepsilon)}\right)^{\frac{2}{n(\varepsilon)}}\right]^{\frac{1}{2}}\right\}$

In [ ]:
T0 = temperatures[0]
epsp0 = epsps[0]
Tm = 1460
order = 9
polyOrderA = order
polyOrderAlpha = order
polyOrdern = order
polyOrderQ = order

In [ ]:
def genParams(params, label, order, srange=None):
    for i in range(order+1):
        if srange==None :
            params.add(label+str(i), value=0)
        else:
            params.add(label+str(i), value=0, min=-srange, max= +srange)

In [ ]:
def polyFunc(eps, opt):
    res = 0
    i = 0
    for k in opt.keys():
        res += opt[k]*eps**i
        i += 1
    return res

Parameter $\alpha$

In [ ]:
n1 = np.empty((0), float)
for strain in strains:
    nn1 = np.empty((0), float)
    for T in temperatures:
        data = identData[(identData[:,0]==strain) & (identData[:,2]==T)]
        a, b = np.polyfit(np.log(data[:,1]), np.log(data[:,3]), 1)
        nn1 = np.append(nn1, 1/a)
    n1 = np.append(n1, nn1.mean())

beta = np.empty((0), float)
for strain in strains:
    beta1 = np.empty((0), float)
    for T in temperatures:
        data = identData[(identData[:,0]==strain) & (identData[:,2]==T)]
        a, b = np.polyfit(np.log(data[:,1]), (data[:,3]), 1)
        beta1 = np.append(beta1, 1/a)
    beta = np.append(beta, beta1.mean())

alpha = beta / n1
#alpha

Parameter $Q$ and $n$

In [ ]:
n = np.empty((0), float)
i = 0
for strain in strains:
    nn1 = np.empty((0), float)
    for T in temperatures:
        data = identData[(identData[:,0]==strain) & (identData[:,2]==T)]
        a, b = np.polyfit(np.log(data[:,1]), np.log(np.sinh(alpha[i]*data[:,3])), 1)
        nn1 = np.append(nn1, 1/a)
    n = np.append(n, nn1.mean())
    i += 1
    
Qf = np.empty((0), float)
i = 0
for strain in strains:
    nn1 = np.empty((0), float)
    for epsp in epsps:
        data = identData[(identData[:,0]==strain) & (identData[:,1]==epsp)]
        a, b =  np.polyfit(1/(data[:,2]+273), np.log(np.sinh(alpha[i]*data[:,3])), 1)
        nn1 = np.append(nn1, a)
    Qf = np.append(Qf, nn1.mean())
    i += 1

R = 8.314
Q = n * R * Qf # For the first range of temperatures
#n, Q

Parameters $\ln(A)$

In [ ]:
lnA = np.empty((0), float)
i = 0
for strain in strains:
    nn1 = np.empty((0), float)
    for T in temperatures:
        data = identData[(identData[:,0]==strain) & (identData[:,2]==T)]
        a, b = np.polyfit(np.log(data[:,1]), np.log(np.sinh(alpha[i]*data[:,3])), 1)
        nn1 = np.append(nn1,(Q[i]/(R*(T+273)) - n[i]*b))
    lnA = np.append(lnA, nn1.mean())
    i += 1
#lnA

Evaluation of $\log(A_i)$ parameters

In [ ]:
lnAi = lmfit.Parameters()
genParams(lnAi, 'A', polyOrderA)
lnAi

In [ ]:
def objLnA(opt):
    return lnA - polyFunc(strains, opt)    

In [ ]:
fitLna = lmfit.minimize(objLnA, lnAi)
fitLna.params

In [ ]:
plt.figure()
plt.plot(strains, lnA, colors[0], marker='s', linestyle = 'none')
plt.plot(strains, polyFunc(strains, fitLna.params), colors[1], label=r'$Fitted$', linewidth = 3)
plt.xlabel(r'$\varepsilon$', fontsize = 16) # Labels the x axis
plt.ylabel(r'$\ln (A)$', fontsize = 16) # Labels the y axis
plt.show()

Evaluation of $\alpha_i$ parameters

In [ ]:
alphai = lmfit.Parameters()
genParams(alphai, 'alpha', polyOrderAlpha, 100)
alphai

In [ ]:
def objAlpha(opt):
    return alpha - polyFunc(strains, opt)    

In [ ]:
fitAlpha = lmfit.minimize(objAlpha, alphai)
fitAlpha.params

In [ ]:
plt.figure()
plt.plot(strains, alpha, colors[0], marker='s', linestyle = 'none')
plt.plot(strains, polyFunc(strains, fitAlpha.params), colors[1], label=r'$Fitted$', linewidth = 3)
plt.xlabel(r'$\varepsilon$', fontsize = 16) # Labels the x axis
plt.ylabel(r'$\alpha$', fontsize = 16) # Labels the y axis
plt.show()

Evaluation of $n_i$ parameters

In [ ]:
ni = lmfit.Parameters()
genParams(ni, 'n', polyOrdern)
ni

In [ ]:
def objN(opt):
    return n - polyFunc(strains, opt)    

In [ ]:
fitN = lmfit.minimize(objN, ni)
fitN.params

In [ ]:
plt.figure()
plt.plot(strains, n, colors[0], marker='s', linestyle = 'none')
plt.plot(strains, polyFunc(strains, fitN.params), colors[1], label=r'$Fitted$', linewidth = 3)
plt.xlabel(r'$\varepsilon$', fontsize = 16) # Labels the x axis
plt.ylabel(r'$n$', fontsize = 16) # Labels the y axis
plt.show()

Evaluation of $Q_i$ parameters

In [ ]:
Qi = lmfit.Parameters()
genParams(Qi, 'Q', polyOrderQ)
Qi

In [ ]:
def objQ(opt):
    return Q - polyFunc(strains, opt)    

In [ ]:
fitQ = lmfit.minimize(objQ, Qi)
fitQ.params

In [ ]:
plt.figure()
plt.plot(strains, Q, colors[0], marker='s', linestyle = 'none')
plt.plot(strains, polyFunc(strains, fitQ.params), colors[1], label=r'$Fitted$', linewidth = 3)
plt.xlabel(r'$\varepsilon$', fontsize = 16) # Labels the x axis
plt.ylabel(r'$Q$', fontsize = 16) # Labels the y axis
plt.show()

In [ ]:
def Arh(eps, epsp, T):
    parAlpha = polyFunc(eps, fitAlpha.params)
    parQ = polyFunc(eps, fitQ.params)
    parN = polyFunc(eps, fitN.params)
    parA = np.exp(polyFunc(eps, fitLna.params))
    parZ = epsp * np.exp(parQ/(R*(T+273)))
    parZA = parZ/parA
    return (1/parAlpha) * np.log(parZA**(1/parN) + (1 + parZA**(2/parN))**(1/2))

In [ ]:
# Plot the curves
from matplotlib.lines import Line2D
def create_dummy_line(**kwds):
    return Line2D([], [], **kwds)

plt.figure(figsize = sbPlotSize(nEpsp))
plt.rc('text', usetex = True)
idx = 1
plt.subplots_adjust(hspace = 0.3)
for epsp in list(epsps):
    xs, ys = sbPlot(nEpsp)
    plt.subplot(xs, ys, idx)
    sbdata = shortData[shortData[:,1]==epsp]
    cl =0
    for temp in list(temperatures):
        sbdata1 = sbdata[sbdata[:,2]==temp]
        plt.plot(sbdata1[:,0], sbdata1[:,3], colors[cl], marker = 's', markersize = 5, linestyle = 'none')
        plt.plot(strains, Arh(strains, epsp, temp), colors[cl], linewidth = 2.5)
        plt.rcParams['xtick.labelsize'] = 16
        plt.rcParams['ytick.labelsize'] = 16
        cl +=1
    plt.xlim(0, 0.7)
    plt.ylim(bottom=0)
    plt.xlabel(r'strain $\varepsilon$', fontsize = 16) # Labels the x axis
    plt.ylabel(r'flow stress $\sigma^y$ (MPa)', fontsize = 16) # Labels the y axis
    plt.title(r'strain rate $\dot{\varepsilon} = ' + str(epsp) + '$ s$^{-1}$', fontsize = 16) # Self explicit command
    idx += 1
    
legendLines = []
cl = 0
for temp in list(temperatures):
    legendLines.append((r'$T=$' + str(int(temp)) + r'$^{\circ}$C', {'color':colors[cl], 'linestyle':'-', 'linewidth':2.5, 'marker':'s'}))
    cl += 1

plt.legend([create_dummy_line(**l[1]) for l in legendLines],[l[0] for l in legendLines], 
           loc = 'upper center', fontsize = 12, ncols = 5, bbox_to_anchor = (0.0, -0.2), shadow = False)

plt.savefig("CompExpAR.svg")
plt.show()

In [ ]:
ARstress = Arh(allData[:,0], allData[:,1], allData[:,2])

In [ ]:
data = np.concatenate((allData[:,0:3],ARstress.reshape((ARstress.shape[0],1))),axis=1)
h5f = h5py.File('AR-6.h5','w')
h5f.create_dataset('data', data = data)
h5f.close()

In [ ]:
EAAR = np.sum(np.abs((allData[:,3] - ARstress)/(allData[:,3])))*100/ARstress.shape[0]
print("EAAR = %g" %(EAAR) + ' %')

In [ ]:
RMSE = np.sqrt(np.sum((allData[:,3] - ARstress)**2)/ARstress.shape[0])
print('RMSE = %g' %(RMSE)+' MPa')

In [ ]:
import math

def truncate(number, decimals=0):
    """
    Returns a value truncated to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer.")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more.")
    elif decimals == 0:
        return math.trunc(number)

    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor

l1=list(fitAlpha.params.values())
l2=list(fitQ.params.values())
l3=list(fitLna.params.values())
l4=list(fitN.params.values())
for i in range(np.max([len(l1),len(l2),len(l3),len(l4)])):
    s = ''
    if i < (len(l1)):
        s += l1[i].name+'='+ str(truncate(l1[i].value,4))
    s+=' & '
    if i < (len(l2)):
        s += l2[i].name+'='+ str(truncate(l2[i].value/1e6,4))
    s+=' & '
    if i < (len(l3)):
        s += l3[i].name+'='+ str(truncate(l3[i].value,4))
    s+=' & '
    if i < (len(l4)):
        s += l4[i].name+'='+ str(truncate(l4[i].value,4))
    s+='\\\\'    
    print(s)